### Long Seq
> Seqence 10개 묶음으로 긴 문장을 Window 이동하면서 데이터 파악

> 다중(Window 이동 O) many-to-many 

> Window Dataset 로직 추가(함수X)

> 다중 네트워크(FC추가) → Class로 구현

In [1]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

# 한 문장으로 처리 할 것
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

sentence = ("if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.")

# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic) 

{',': 0, 's': 1, 'a': 2, 'c': 3, 'b': 4, 'd': 5, 'i': 6, 'k': 7, 'o': 8, 'g': 9, 'l': 10, "'": 11, '.': 12, 'u': 13, 'y': 14, 'h': 15, 'f': 16, 't': 17, 'r': 18, 'p': 19, 'w': 20, 'e': 21, 'n': 22, ' ': 23, 'm': 24}


In [2]:
dic_size = len(char_dic)
dic_size

25

### 데이터 처리 : Window DataSet 작업

In [3]:
len(sentence)

180

In [4]:
sequence_length = 10  # Any arbitrary number

# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):   # 0 ~ 170
    x_str = sentence[i:i + sequence_length]                  # X : 170개의 Window단위(10) 데이터 생김  [ 170, 10, 단어차원 ]
    y_str = sentence[i + 1: i + sequence_length + 1]    # Y : 동일함                                                      [ 170, 10 ]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [5]:
print(X.shape) 
print(Y.shape)

torch.Size([170, 10, 25])
torch.Size([170, 10])


#### 단어사전/원핫 특징을 제외하고는 Window이동에 따른 데이터 셋 [170개, 10개묶음] 준비는 X, Y 동일
- ( Batch = 1 )
- 전체 row (Window 이동 횟수) = 170
- Sequence (Windiow) : 10
- input_size (특징차원, 단어사전/원핫, input node) : 25
- output_size : 10 (to-many 이므로 sequence 길이와 같음)

In [6]:
X[0] # Window 1개 

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,

In [22]:
# hyper parameters
input_size       = 25 # dic_size
hidden_size    = 25 # 아무 값이 나 넣어봄  / 같은 값 넣어주는게 성능이 좋았음(줄여도  ok, 키우면 이상)
output_dim    = 25 # dic_size
learning_rate = 0.1

# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers): # ( 25, 25, 25, layer )
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True) # hidden이 다음 input이 되면 됨
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True) 

    def forward(self, x):
        x, _status = self.rnn(x)
        print("                                        순서 Row, Window, InputFeature")
        print("rnn후  output :", x.shape)
        print("fc 후  output :", x.shape)
        return x

net = Net(dic_size, hidden_size, dic_size, 2)
print("\n", net(X).shape)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

                                        순서 Row, Window, InputFeature
rnn후  output : torch.Size([170, 10, 25])
fc 후  output : torch.Size([170, 10, 25])

 torch.Size([170, 10, 25])


In [8]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    print("\n▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶noutput: ", outputs.shape, "▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶")  
    print("outputs.view(-1, dic_size) : ", outputs.view(-1, dic_size).shape)
    print("Y.view(-1) : ", Y.shape,"→",Y.view(-1).shape)
    
    # 단어사전/원한 특징 차원값을 제외하고는 X, Y 모두 'view(-1' 하여 3D → 2D 로 변환 후 비교
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # X: [170, 10, 25]→[1700, 25], 
                                                                                                    # Y:  [170,10]       →[1700]
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        print("epoch[",i,"]", j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

rnn후  output : torch.Size([170, 10, 25])  _status : torch.Size([2, 170, 25])
fc 후  output : torch.Size([170, 10, 25])

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶noutput:  torch.Size([170, 10, 25]) ▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
outputs.view(-1, dic_size) :  torch.Size([1700, 25])
Y.view(-1) :  torch.Size([170, 10]) → torch.Size([1700])
epoch[ 0 ] 0 .uruuuuk.. 3.269895076751709
epoch[ 0 ] 1 .cuuurk... 3.269895076751709
epoch[ 0 ] 2 .uuuuk.... 3.269895076751709
epoch[ 0 ] 3 .uuuk..... 3.269895076751709
epoch[ 0 ] 4 .uuk.....u 3.269895076751709
epoch[ 0 ] 5 cuk.....uu 3.269895076751709
epoch[ 0 ] 6 .uru...uuu 3.269895076751709
epoch[ 0 ] 7 kcuu..uuuu 3.269895076751709
epoch[ 0 ] 8 cuuu.uuuu. 3.269895076751709
epoch[ 0 ] 9 .uuuuuuu.k 3.269895076751709
epoch[ 0 ] 10 .uuuuuu.kc 3.269895076751709
epoch[ 0 ] 11 .uuuuu.kcu 3.269895076751709
epoch[ 0 ] 12 .uuuu..cuk 3.269895076751709
epoch[ 0 ] 13 .uuu.kcukr 3.269895076751709
epoch[ 0 ] 14 .uu.kkukrk 3.269895076751709
epoch[ 0 ] 15 cukukukrk. 3.269895076751709
epoch[ 0 ] 16 c

### 한칸 뒤부터 잘 예측하였음

In [51]:
# sentence = ("if you want to build a ship, don't drum up people together to "
#             "collect wood and don't assign them tasks and work, but rather "
#             "teach them to long for the endless immensity of the sea.")

### Clean 소스 버전

In [109]:
# # hyper parameters
# input_size = dic_size
# hidden_size = dic_size          # 아무 값이 나 넣어봄  / 같은 값 넣어주는게 성능이 좋았음(줄여도  ok, 키우면 이상)
# output_dim = dic_size
# learning_rate = 0.1

# # declare RNN + FC
# class Net(torch.nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim, layers):
#         super(Net, self).__init__()
#         self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
#         self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

#     def forward(self, x):
#         x, _status = self.rnn(x)
#         x = self.fc(x)
#         return x

# net = Net(dic_size, hidden_size, dic_size, 2)

# # loss & optimizer setting
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), learning_rate)

In [52]:
# # start training
# for i in range(100):
#     optimizer.zero_grad()
#     outputs = net(X)
    
#     loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) 
#     loss.backward()
#     optimizer.step()

#     results = outputs.argmax(dim=2)
#     predict_str = ""
#     for j, result in enumerate(results):
#         print(i, j, ''.join([char_set[t] for t in result]), loss.item())
#         if j == 0:
#             predict_str += ''.join([char_set[t] for t in result])
#         else:
#             predict_str += char_set[result[-1]]

#     print(predict_str)